# Загрузка данных в Qdrant

Этот ноутбук:
1. Удаляет все существующие документы из Qdrant
2. Загружает данные из JSON файла с сообщениями
3. Добавляет документы в векторную базу данных
4. Проверяет корректность загрузки


## 1. Импорты и настройка


In [ ]:
import json
import logging
import sys
from pathlib import Path
from datetime import datetime

# Добавляем путь к проекту
project_root = Path("/srv/nlp1/eval_dir/T-bank_NLP_")
sys.path.insert(0, str(project_root))

# Настройка логирования
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)

print("=" * 80)
print(f"📦 ЗАГРУЗКА ДАННЫХ В QDRANT - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 80)


## 2. Инициализация RetrieverService


In [ ]:
from tplexity.retriever.retriever_service import RetrieverService

print("🔄 Инициализация RetrieverService...")
service = RetrieverService()
print("✅ RetrieverService инициализирован")


## 3. Проверка количества документов (до очистки)


In [ ]:
count_before = await service.vector_search.client.count(
    collection_name=service.vector_search.collection_name
)
print(f"📊 Документов в коллекции '{service.vector_search.collection_name}' (до очистки): {count_before.count}")


## 4. Удаление всех документов


In [ ]:
print("⚠️  Удаление всех документов из Qdrant...")
await service.delete_all_documents()
print("✅ Все документы удалены")

# Проверяем, что коллекция пуста
count_after_delete = await service.vector_search.client.count(
    collection_name=service.vector_search.collection_name
)
print(f"📊 Документов после очистки: {count_after_delete.count}")


## 5. Загрузка данных из JSON


In [ ]:
# Путь к JSON файлу с сообщениями
messages_path = project_root / "src/eval/eval_data/messages_diverse_1000posts_all_channels.json"

print(f"🔄 Загрузка сообщений из {messages_path.name}...")
with open(messages_path, "r", encoding="utf-8") as f:
    messages = json.load(f)

print(f"✅ Загружено {len(messages)} сообщений из файла")
print(f"\n📋 Пример первого сообщения:")
print(f"  ID: {messages[0]['id']}")
print(f"  Channel ID: {messages[0]['channel_id']}")
print(f"  Date: {messages[0].get('date', 'N/A')}")
print(f"  Text: {messages[0].get('text', '')[:100]}...")


## 6. Подготовка данных для индексации


In [ ]:
print("🔄 Подготовка документов для индексации...")

documents = []
metadatas = []

for msg in messages:
    text = msg.get("text", "").strip()
    if not text:
        continue
    
    documents.append(text)
    
    # Метаданные (составной ID для связи с ground truth)
    metadata = {
        "doc_id": f"{msg['channel_id']}_{msg['id']}",  # Составной ID
        "message_id": msg["id"],
        "channel_id": msg["channel_id"],
        "date": msg.get("date", ""),
        "link": msg.get("link", ""),
    }
    metadatas.append(metadata)

print(f"✅ Подготовлено {len(documents)} документов для индексации")
print(f"📊 Пропущено {len(messages) - len(documents)} пустых сообщений")


## 7. Добавление документов в Qdrant


In [ ]:
print("🔄 Загрузка документов в Qdrant...")
print(f"⏳ Это может занять несколько минут...")

# Добавляем документы (UUID будут сгенерированы автоматически)
await service.add_documents(documents=documents, metadatas=metadatas)

print("✅ Документы успешно добавлены в Qdrant")


## 8. Проверка количества документов (после загрузки)


In [ ]:
count_after_load = await service.vector_search.client.count(
    collection_name=service.vector_search.collection_name
)

print("=" * 80)
print("📊 ИТОГОВАЯ СТАТИСТИКА")
print("=" * 80)
print(f"📈 Документов до очистки:     {count_before.count}")
print(f"🗑️  Документов после очистки:  {count_after_delete.count}")
print(f"✅ Документов после загрузки:  {count_after_load.count}")
print(f"📦 Добавлено документов:       {len(documents)}")
print("=" * 80)

if count_after_load.count == len(documents):
    print("✅ Все документы успешно загружены!")
else:
    print(f"⚠️  Расхождение: ожидалось {len(documents)}, загружено {count_after_load.count}")


## 9. Тестовый поиск


In [ ]:
print("\n🔍 Тестовый поиск для проверки работоспособности...\n")

test_query = "инвестиции акции"
results = await service.search(test_query, top_k=5, top_n=3, use_rerank=False)

print(f"Запрос: '{test_query}'")
print(f"Найдено результатов: {len(results)}\n")

for i, (doc_id, score, text, metadata) in enumerate(results, 1):
    print(f"Результат {i}:")
    print(f"  Doc ID: {doc_id}")
    print(f"  Score: {score:.3f}")
    print(f"  Text: {text[:150]}...")
    print(f"  Metadata doc_id: {metadata.get('doc_id', 'N/A')}")
    print(f"  Link: {metadata.get('link', 'N/A')}")
    print()


## 10. Проверка конкретного документа по ground truth ID


In [ ]:
# Проверяем, что можем найти конкретный документ
print("\n🔍 Проверка поиска конкретного документа...\n")

# Берем первое сообщение из нашего датасета
first_msg = messages[0]
expected_doc_id = f"{first_msg['channel_id']}_{first_msg['id']}"

print(f"Ожидаемый doc_id: {expected_doc_id}")
print(f"Текст документа: {first_msg.get('text', '')[:150]}...\n")

# Ищем по тексту из этого документа
search_text = first_msg.get('text', '')[:100]  # Берем первые 100 символов
results = await service.search(search_text, top_k=5, top_n=5, use_rerank=False)

print(f"Результаты поиска (топ-5):")
found = False
for i, (doc_id, score, text, metadata) in enumerate(results, 1):
    metadata_doc_id = metadata.get('doc_id', '')
    is_target = metadata_doc_id == expected_doc_id
    marker = "✅ [ЦЕЛЕВОЙ ДОКУМЕНТ]" if is_target else ""
    
    print(f"  {i}. doc_id: {metadata_doc_id}, Score: {score:.3f} {marker}")
    
    if is_target:
        found = True

print()
if found:
    print("✅ Целевой документ найден! Система работает корректно.")
else:
    print("⚠️  Целевой документ не найден в топ-5. Это может быть нормально.")


## Готово! 🎉

Данные успешно загружены в Qdrant. Теперь можно запускать evaluation ноутбуки для оценки качества поиска.
